# TCP/UDP Communication System - Farnsworth Fusor Analysis

## Overview
This notebook provides a comprehensive analysis of the TCP/UDP communication system for the Farnsworth Fusor project.

## Purpose
The TCP/UDP system implements bidirectional communication:
- **TCP for Commands**: Host sends control commands to target (LED_ON, LED_OFF, SET_VOLTAGE, etc.)
- **TCP for Data**: Target sends sensor data to host every 1 second
- **UDP for Status/Heartbeat**: Bidirectional status messages for system health monitoring
- **Bidirectional Communication**: Complete implementation of command and data exchange

## Architecture Benefits
- **TCP Commands**: Reliable, connection-oriented command transmission
- **TCP Data**: Reliable sensor data transmission
- **UDP Status**: Lightweight, fast status/heartbeat messages
- **No SSH Required**: Simplified network setup without SSH dependencies


## System Architecture

### Visual Summary
```
┌─────────────────┐    TCP Commands     ┌─────────────────┐
│                 │ ──────────────────► │                 │
│   HOST MACHINE  │    (Port 2222)     │  TARGET (RPi)   │
│                 │ ◄────────────────── │                 │
└─────────────────┘    TCP Data        └─────────────────┘
        │             (Port 12345)              │
        │                                        │
        │ UDP Status (Port 8888/8889)            │
        │ ◄──────────────────────────────────► │
        │                                        │
        │ TCP Command Client (2222)              │ TCP Command Server (2222)
        │ TCP Data Server (12345)                │ TCP Data Client (12345)
        │ UDP Status Client/Receiver             │ UDP Status Sender/Receiver
        │                                        │
        │ Commands: LED_ON, LED_OFF,             │ Data: GPIO_INPUT, ADC_DATA
        │          SET_VOLTAGE, SET_PUMP_POWER   │ Status: Heartbeat messages
```

### Communication Flow
1. **Host → Target (TCP Commands)**: Control commands sent via TCP on port 2222
2. **Target → Host (TCP Data)**: Sensor data sent via TCP on port 12345 every 1 second
3. **Bidirectional (UDP Status)**: Status/heartbeat messages on ports 8888/8889
4. **Complete Data Exchange**: Full bidirectional communication as required


## Host Codebase Components

### Core Files

#### 1. `host_main.py`
- **Purpose**: Single entry point with Tkinter control panel
- **Functionality**: GUI application that sends commands to target
- **Key Features**:
  - Control panel with buttons for all commands
  - TCP command client to send commands to target
  - TCP data server to receive read-only data from target
  - UDP status communication for system health
  - Real-time data display showing all responses

#### 2. `tcp_command_client.py`
- **Purpose**: TCP client for sending commands to target
- **Functionality**: Sends commands and receives responses
- **Key Features**:
  - Connects to target TCP command server (port 2222)
  - Sends commands: LED_ON, LED_OFF, SET_VOLTAGE, etc.
  - Handles connection errors and reconnection
  - Returns command responses from target

#### 3. `udp_status_client.py`
- **Purpose**: UDP client/server for status/heartbeat communication
- **Functionality**: Sends and receives UDP status messages
  - `UDPStatusClient`: Sends status to target (port 8889)
  - `UDPStatusReceiver`: Receives status from target (port 8888)

#### 4. `host_control.py`
- **Purpose**: Command-line interface for host control
- **Functionality**: Alternative to GUI for sending commands
- **Key Features**:
  - Interactive command prompt
  - Same TCP command functionality as GUI
  - Useful for scripting or headless operation


## Target Codebase Components

### Core Files

#### 1. `target_main.py`
- **Purpose**: Main entry point for target system
- **Functionality**: Orchestrates TCP command server and data client
- **Key Features**:
  - Starts TCP command server on port 2222
  - Initializes TCP data client to send data to host
  - Sets up UDP status communication
  - Handles graceful shutdown
  - Runs at boot via systemd service

#### 2. `tcp_command_server.py`
- **Purpose**: TCP server for receiving commands from host
- **Functionality**: Handles incoming TCP commands
- **Key Features**:
  - Listens for commands on port 2222
  - Processes LED_ON/LED_OFF, SET_VOLTAGE, SET_PUMP_POWER, MOVE_VAR commands
  - Controls GPIO pins (LED control)
  - Integrates with motor control and ADC systems
  - Sends responses back to host

#### 3. `tcp_client.py`
- **Purpose**: TCP client for sending sensor data to host
- **Functionality**: Sends data to host every 1 second
- **Key Features**:
  - Connects to host TCP server (port 12345)
  - Sends GPIO input and ADC data
  - Handles connection errors gracefully
  - Automatic reconnection on failures

#### 4. `udp_status_server.py`
- **Purpose**: UDP server for status/heartbeat communication
- **Functionality**: Sends and receives UDP status messages
  - `UDPStatusSender`: Sends status to host (port 8888)
  - `UDPStatusReceiver`: Receives status from host (port 8889)


## Network Configuration

### Default Settings
- **Host IP**: 192.168.0.1 (default)
- **Target IP**: 192.168.0.2 (default)
- **Subnet**: 255.255.255.0
- **TCP Command Port**: 2222 (Target server)
- **TCP Data Port**: 12345 (Host server)
- **UDP Status Ports**: 8888 (Host receive), 8889 (Target receive)
- **No Gateway/DNS**: Manual static IP configuration

### Configuration Methods
1. **Command Line Arguments**:
   ```bash
   # Host
   python src/Host_Codebase/host_main.py --target-ip 192.168.0.2
   
   # Target
   python src/Target_Codebase/target_main.py --host 192.168.0.1
   ```

2. **Environment Variables** (Target):
   - `FUSOR_HOST_IP`: Host IP address

3. **Static IP Configuration**:
   - Windows Host: Network settings → Ethernet properties → Manual IP
   - RPi Target: nmcli commands for static IP


## Data Flow Analysis

### Command Flow (Host → Target)
1. User clicks button in `host_main.py` control panel
2. Host sends TCP command via `TCPCommandClient` to target (port 2222)
3. Target receives command via `TCPCommandServer`
4. Target executes action (GPIO control, motor control, etc.)
5. Target sends response back via TCP
6. Host receives response and displays in data display area

### Data Flow (Target → Host)
1. Target reads GPIO input pin periodically
2. Target reads ADC sensor data (if enabled)
3. Target formats data: `GPIO_INPUT:0` or `ADC_DATA:512,256,...`
4. Target sends data via `TargetTCPCommunicator` to host (port 12345)
5. Host receives data via TCP data server in `host_main.py`
6. Host displays read-only data in data display area

### Status Flow (Bidirectional UDP)
1. Host sends status via `UDPStatusClient` to target (port 8889)
2. Target receives status via `UDPStatusReceiver`
3. Target sends status via `UDPStatusSender` to host (port 8888)
4. Host receives status via `UDPStatusReceiver`
5. Both systems display status for health monitoring

### Periodic Data Transmission
- **Interval**: Every 1 second
- **Data Types**: GPIO input, ADC readings
- **Format**: Structured messages with delimiters
- **Error Handling**: Automatic reconnection on failures


## Implementation Status

### ✅ **Implemented Features**
- TCP command server for reliable command reception (port 2222)
- TCP data client for efficient data transmission (port 12345)
- UDP status/heartbeat communication (ports 8888/8889)
- GPIO control for LED operations
- ADC integration for sensor data
- Motor control integration
- Configurable network settings
- Error handling and reconnection logic
- Single entry point with control panel (`host_main.py`)
- Real-time data display
- Command-line interface (`host_control.py`)

### 🎯 **Project Goals Achieved**
- **TCP/UDP Communication**: Complete bidirectional communication
- **Command Control**: LED_ON/LED_OFF, SET_VOLTAGE, SET_PUMP_POWER, MOVE_VAR functionality
- **Data Transmission**: Continuous sensor data flow every 1 second
- **Network Flexibility**: Adaptable to different network configurations
- **User Interface**: Control panel with all commands accessible
- **Read-only Data**: Target sends sensor data for display on host

The system is ready for testing and provides a solid foundation for the complete Fusor control system.
